<a href="https://colab.research.google.com/github/sgbyteninja/sentiment_analysis_customer_reviews/blob/main/RoBERTa_training_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports


In [44]:
!pip install datasets
import pandas as pd
from datasets import Dataset
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AutoConfig, TrainingArguments, Trainer, EarlyStoppingCallback, AutoModelForSequenceClassification, AutoTokenizer
from torch.utils.data import DataLoader
from google.colab import drive
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


# Tokenisation

In [45]:
# URLs of the CSV files from your GitHub repository
train_url = 'https://raw.githubusercontent.com/sgbyteninja/sentiment_analysis_customer_reviews/refs/heads/main/train_data.csv'
test_url = 'https://raw.githubusercontent.com/sgbyteninja/sentiment_analysis_customer_reviews/refs/heads/main/test_data.csv'
val_url = 'https://raw.githubusercontent.com/sgbyteninja/sentiment_analysis_customer_reviews/refs/heads/main/val_data.csv'

# Load the CSV files into pandas DataFrames
train_df = pd.read_csv(train_url)
test_df = pd.read_csv(test_url)
val_df = pd.read_csv(val_url)


In [46]:
# Convert train_df to Hugging Face Dataset format
train_df = Dataset.from_pandas(train_df)
val_df = Dataset.from_pandas(val_df)
test_df = Dataset.from_pandas(test_df)
# Load the RoBERTa tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

In [47]:
# Tokenize the review column
def tokenize_function(examples):
    return tokenizer(examples['review'], padding="max_length", truncation=True)

In [48]:
# Tokenize datasets
train_df = train_df.map(tokenize_function, batched=True)
val_df = val_df.map(tokenize_function, batched=True)
test_df = test_df.map(tokenize_function, batched=True)

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [49]:
# Check if the tokenization worked properly
print(train_df[0])  # First entry train_df
print(val_df[0])    # First entry val_df
print(test_df[0])   # first entry test_df

{'review': "A very nice pizza.  I haven't made up my mind which is more authentic, NYPD or Ray's.  Both, have the thin NY style crust, but NYPD makes me feel like I'm waiting for an E train to Queens; whereas Ray's feels more upper east side-ish.", 'label': 0, 'input_ids': [0, 250, 182, 2579, 9366, 4, 1437, 38, 2220, 75, 156, 62, 127, 1508, 61, 16, 55, 12757, 6, 20283, 50, 4622, 18, 4, 1437, 1868, 6, 33, 5, 7174, 5300, 2496, 22196, 6, 53, 20283, 817, 162, 619, 101, 38, 437, 2445, 13, 41, 381, 2341, 7, 12446, 131, 9641, 4622, 18, 2653, 55, 2853, 3017, 526, 12, 1173, 4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [50]:
test_df

Dataset({
    features: ['review', 'label', 'input_ids', 'attention_mask'],
    num_rows: 1000
})

In [51]:
# Assure the compatibility with PyTorch
train_df.set_format("torch", columns=["input_ids", "attention_mask", "label"])
val_df.set_format("torch", columns=["input_ids", "attention_mask", "label"])
test_df.set_format("torch", columns=["input_ids", "attention_mask", "label"])

# Model and Configuration Setup


In [52]:
# extract the number of names an classes
num_labels = len(set(train_df["label"]))
id2label = {i: str(i) for i in range(num_labels)}

In [53]:
# Configuration of the RoBERTa-model
config = AutoConfig.from_pretrained("roberta-base")
config.update({"id2label": id2label, "num_labels": num_labels})

In [54]:
# Load the RoBERTa-model
model = RobertaForSequenceClassification.from_pretrained("roberta-base", config=config)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Defining the Metrics for the Model Evaluation


In [55]:
def compute_metrics(p):
    predictions, labels = p
    predictions = predictions.argmax(axis=-1)  # For multi-class classification

    # Compute precision, recall, and f1-score for each class
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average='weighted'  # Use 'micro' or 'macro' for other types of averaging
    )

    accuracy = accuracy_score(labels, predictions)

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

# Define the Parameters for Training


In [58]:
training_args = TrainingArguments(
    output_dir="./roberta_sentiment",
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    learning_rate=5e-5,
    weight_decay=0.01,
    warmup_steps=500,
    save_strategy="epoch",
    load_best_model_at_end=True,
    save_total_limit=2,
    report_to="none",
    metric_for_best_model="eval_loss",  # using eval_loss for EarlyStopping
    greater_is_better=False
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


# Setup the Trainer and train the Model


In [27]:
drive.mount('/content/drive')

Mounted at /content/drive


In [59]:
# Early Stopping helps prevent overfitting by stopping training when the validation loss stops improving.
# This ensures that the model does not continue learning patterns that do not generalize to unseen data.

# The parameter early_stopping_patience=3 means that training stops if there is no improvement in evaluation loss
# for 3 consecutive epochs. This prevents wasting computational resources on unnecessary training.

# Early stopping is particularly useful when training deep learning models, as they can easily overfit if trained for too long.
# By stopping early, we keep the model at its best generalization point.

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_df,
    eval_dataset=val_df,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

<ipython-input-59-2c4c052f8ab4>:10: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [60]:
# train the model
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.454900,0.477656,0.807000,0.823510,0.807000,0.810960
2,0.435000,0.462104,0.839000,0.824984,0.839000,0.828665
3,0.569800,0.573540,0.843000,0.841549,0.843000,0.840929
4,0.311700,0.684049,0.850000,0.850986,0.850000,0.850420
5,0.062000,0.817452,0.851000,0.847902,0.851000,0.848640


TrainOutput(global_step=5000, training_loss=0.5308254918649793, metrics={'train_runtime': 4034.2623, 'train_samples_per_second': 9.915, 'train_steps_per_second': 1.239, 'total_flos': 1.128021098496e+16, 'train_loss': 0.5308254918649793, 'epoch': 5.0})

In [61]:
# Saving the trained model on Google Drive
# Path for saving in Google Drive
model_path = "/content/drive/MyDrive/Roberta_for_sentiment_analysis"

# Save the trained model
trainer.save_model(model_path)

# Save the tokenizer
tokenizer.save_pretrained(model_path)


('/content/drive/MyDrive/Roberta_for_sentiment_analysis/tokenizer_config.json',
 '/content/drive/MyDrive/Roberta_for_sentiment_analysis/special_tokens_map.json',
 '/content/drive/MyDrive/Roberta_for_sentiment_analysis/vocab.json',
 '/content/drive/MyDrive/Roberta_for_sentiment_analysis/merges.txt',
 '/content/drive/MyDrive/Roberta_for_sentiment_analysis/added_tokens.json')

In [ ]:
# Path to your saved model and tokenizer
model_path = "/content/drive/MyDrive/Roberta_for_sentiment_analysis"

# Load the model
model = RobertaForSequenceClassification.from_pretrained(model_path)

# Load the tokenizer
tokenizer = RobertaTokenizer.from_pretrained(model_path)

# Evaluation of the Model


In [62]:
# Evaluate the model
results = trainer.evaluate()

# Print evaluation results
print(results)


{'eval_loss': 0.4621041417121887, 'eval_accuracy': 0.839, 'eval_precision': 0.8249835968582958, 'eval_recall': 0.839, 'eval_f1': 0.8286650784196804, 'eval_runtime': 27.7338, 'eval_samples_per_second': 36.057, 'eval_steps_per_second': 4.507, 'epoch': 5.0}


In [63]:
results

{'eval_loss': 0.4621041417121887,
 'eval_accuracy': 0.839,
 'eval_precision': 0.8249835968582958,
 'eval_recall': 0.839,
 'eval_f1': 0.8286650784196804,
 'eval_runtime': 27.7338,
 'eval_samples_per_second': 36.057,
 'eval_steps_per_second': 4.507,
 'epoch': 5.0}